## Importing libraries

In [1]:
import pandas as pd
import os
import spacy
from spacy.tokenizer import Tokenizer

import re
import string
import unicodedata


In [2]:
from streamlit_agraph import agraph, Node, Edge, Config

In [6]:
import spacy
from scispacy.abbreviation import AbbreviationDetector


nlp = spacy.load("en_core_web_sm")

def expand_abbreviation(text):
    nlp.add_pipe("abbreviation_detector")
    doc = nlp(text)
    for abrv in doc._.abbreviations:
        text = text.replace(abrv.text,abrv._.long_form.text)
    return text


# Add the abbreviation pipe to the spacy pipeline.
text = "AUSTRALIAN FOREIGN SHIP BAN ENDS BUT NSW PORTS HIT Tug crews in New South Wales (NSW), \
Victoria and Western Australia yesterday lifted their ban on foreign-flag ships carrying containers but NSW ports are still being disrupted by a separate dispute, \
shipping sources said. The ban, imposed a week ago over a pay claim, had prevented the movement in or out of port of nearly 20 vessels, they said. The pay dispute went before\
 a hearing of the Arbitration Commission today. Meanwhile, disruption began today to cargo handling in the ports of Sydney, Newcastle and Port Kembla, they said. The industrial\
  action at the NSW ports is part of the week of action called by the NSW Trades and Labour Council to protest changes to the state workers compensation laws. The shipping sources \
  said the various port unions appear to be taking it in turn to work for a short time at the start of each shift and then to walk off. Cargo handling in the ports has been disrupted, \
  with container movements most affected, but has not stopped altogether, they said. \
They said they could not say how long the disruption will go on and what effect it will have on shipping movements."

print(expand_abbreviation(text))






AUSTRALIAN FOREIGN SHIP BAN ENDS BUT New South Wales PORTS HIT Tug crews in New South Wales (New South Wales), Victoria and Western Australia yesterday lifted their ban on foreign-flag ships carrying containers but New South Wales ports are still being disrupted by a separate dispute, shipping sources said. The ban, imposed a week ago over a pay claim, had prevented the movement in or out of port of nearly 20 vessels, they said. The pay dispute went before a hearing of the Arbitration Commission today. Meanwhile, disruption began today to cargo handling in the ports of Sydney, Newcastle and Port Kembla, they said. The industrial  action at the New South Wales ports is part of the week of action called by the New South Wales Trades and Labour Council to protest changes to the state workers compensation laws. The shipping sources   said the various port unions appear to be taking it in turn to work for a short time at the start of each shift and then to walk off. Cargo handling in the po

In [20]:


from spacy import load as spacy_load
from spikex.pipes import AbbrX

ModuleNotFoundError: No module named 'spikex.pipes'

In [15]:
import spacy
from scispacy.abbreviation import AbbreviationDetector

nlp = spacy.load("en_core_web_sm")

abbreviation_pipe = AbbreviationDetector(nlp)
nlp.add_pipe(abbreviation_pipe)



ValueError: [E966] `nlp.add_pipe` now takes the string name of the registered component factory, not a callable component. Expected string, but got <scispacy.abbreviation.AbbreviationDetector object at 0x000002424D2BCEB0> (name: 'None').

- If you created your component with `nlp.create_pipe('name')`: remove nlp.create_pipe and call `nlp.add_pipe('name')` instead.

- If you passed in a component like `TextCategorizer()`: call `nlp.add_pipe` with the string name instead, e.g. `nlp.add_pipe('textcat')`.

- If you're using a custom component: Add the decorator `@Language.component` (for function components) or `@Language.factory` (for class components / factories) to your custom component and assign it a name, e.g. `@Language.component('your_name')`. You can then run `nlp.add_pipe('your_name')` to add it to the pipeline.

ModuleNotFoundError: No module named 'en_core_sci_lg'

In [18]:
def find_obj(doc):
    obj_dep= ['dobj', 'pobj', 'iobj', 'obj', 'obl']
    obj = None
    for token in doc.rights:
        if token.dep_ != 'punct':  
            if token.dep_ in obj_dep:
                obj = token
            else:
                if token.dep_ == 'ccomp':
                    obj = find_sub(token)
                for right in token.rights:
                    obj = right
        else:
            break
    for token in doc.lefts:
        if token.dep_ == 'ccomp':
            obj = find_sub(token)
#     print(obj)
    if obj != None:
        if obj.nbor().pos_=="CCONJ":
            obj = get_conj_words(obj)
        else:
            obj = get_full_word(obj)
    return obj
            
def is_passive(token):
    if token.dep_.endswith('pass'): # noun
        return True

In [19]:
def find_sub(pred):
    if len(list(pred.lefts))>0:
        for token in pred.lefts:
            if token.dep_ in ['nsubj', 'nsubjpass']:
                return token
#     if pred.head != pred and not is_passive(pred): 
#         print("Head of ",pred,' is ',pred.head)
    
#         return find_sub(pred.head)
#     else:
#         return None 
    else:
        if pred.nbor(-1).dep_ != 'punct':
            return pred.nbor(-1)
        else:
            return pred.nbor(-2)

In [20]:
def find_rel(doc):
    df = pd.DataFrame()
    subject =[]
    target = []
    relation = []
    for token in doc:
        if token.dep_== 'ccomp' or token.dep_=='ROOT' or token.pos_ =='VERB':
            sub = find_sub(token)
            if token.nbor().dep_ == 'agent':
                rel = " ".join([token.text,token.nbor().text])
            else:
                rel = token.text
            if sub is None:
                continue
            sub= get_full_word(sub)
            obj = find_obj(token)
            if obj is None:
                continue
#             obj = get_full_word(obj)
            subject.append(sub)
            target.append(obj)
            relation.append(rel)
    input_df = pd.DataFrame({'source':subject, 'relation':relation, 'target': target})
    print(input_df)
    return input_df

In [21]:
df = pd.read_excel("..//Assets//"+"preprocessed_text.xlsx")



In [22]:
df.iloc[0,2]

'Mounting trade friction between the U.S. And Japan has raised fears among many of Asias exporting nations that the row could inflict far-reaching economic damage, businessmen and officials said. They told Reuter correspondents in Asian capitals a U.S. Move against Japan might boost protectionist sentiment in the U.S. And lead to curbs on American imports of their products. But some exporters said that while the conflict would hurt them in the long-run, in the short-term Tokyos loss might be their gain. The U.S. Has said it will impose 300 mln dlrs of tariffs on imports of Japanese electronics goods on April 17, in retaliation for Japans alleged failure to stick to a pact not to sell semiconductors on world markets at below cost. Unofficial Japanese estimates put the impact of the tariffs at 10 billion dlrs and spokesmen for major electronics firms said they would virtually halt exports of products hit by the new taxes. "We would not be able to do business," said a spokesman for leadin

In [23]:
doc = nlp(df.iloc[0,2])


In [24]:
find_rel(doc)

IndexError: [E042] Error accessing `doc[0].nbor(-1)`, for doc of length 799.

In [4]:
CONTRACTION_MAP = {
"ain't": "is not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he he will have",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how is",
"I'd": "I would",
"I'd've": "I would have",
"I'll": "I will",
"I'll've": "I will have",
"I'm": "I am",
"I've": "I have",
"i'd": "i would",
"i'd've": "i would have",
"i'll": "i will",
"i'll've": "i will have",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "it will have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she would",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as",
"that'd": "that would",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there would",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they would",
"they'd've": "they would have",
"they'll": "they will",
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what'll've": "what will have",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"when's": "when is",
"when've": "when have",
"where'd": "where did",
"where's": "where is",
"where've": "where have",
"who'll": "who will",
"who'll've": "who will have",
"who's": "who is",
"who've": "who have",
"why's": "why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you would",
"you'd've": "you would have",
"you'll": "you will",
"you'll've": "you will have",
"you're": "you are",
"you've": "you have"
}

In [5]:
os.listdir('../Assets')


['input_text.csv', 'preprocessed_text.xlsx', 'reutersNLTK.xlsx']

### Reading the first text of the dataframe

In [6]:
text_list = []
df = pd.read_excel('../Assets/reutersNLTK.xlsx')
first_text = df.iloc[0,2]

first_text

'ASIAN EXPORTERS FEAR DAMAGE FROM U.S.-JAPAN RIFT\n  Mounting trade friction between the\n  U.S. And Japan has raised fears among many of Asia\'s exporting\n  nations that the row could inflict far-reaching economic\n  damage, businessmen and officials said.\n      They told Reuter correspondents in Asian capitals a U.S.\n  Move against Japan might boost protectionist sentiment in the\n  U.S. And lead to curbs on American imports of their products.\n      But some exporters said that while the conflict would hurt\n  them in the long-run, in the short-term Tokyo\'s loss might be\n  their gain.\n      The U.S. Has said it will impose 300 mln dlrs of tariffs on\n  imports of Japanese electronics goods on April 17, in\n  retaliation for Japan\'s alleged failure to stick to a pact not\n  to sell semiconductors on world markets at below cost.\n      Unofficial Japanese estimates put the impact of the tariffs\n  at 10 billion dlrs and spokesmen for major electronics firms\n  said they would v

In [21]:
text_list = df.iloc[:,2].values
text_list

array(['ASIAN EXPORTERS FEAR DAMAGE FROM U.S.-JAPAN RIFT\n  Mounting trade friction between the\n  U.S. And Japan has raised fears among many of Asia\'s exporting\n  nations that the row could inflict far-reaching economic\n  damage, businessmen and officials said.\n      They told Reuter correspondents in Asian capitals a U.S.\n  Move against Japan might boost protectionist sentiment in the\n  U.S. And lead to curbs on American imports of their products.\n      But some exporters said that while the conflict would hurt\n  them in the long-run, in the short-term Tokyo\'s loss might be\n  their gain.\n      The U.S. Has said it will impose 300 mln dlrs of tariffs on\n  imports of Japanese electronics goods on April 17, in\n  retaliation for Japan\'s alleged failure to stick to a pact not\n  to sell semiconductors on world markets at below cost.\n      Unofficial Japanese estimates put the impact of the tariffs\n  at 10 billion dlrs and spokesmen for major electronics firms\n  said they 

In [13]:
normalized_text = ' '.join(re.sub(' +', ' ',first_text.replace('\'s','').replace('\'t','').replace('&lt;','').replace(">","")).split('\n')[1:]).strip().replace('  ',' ')
normalized_text

'Mounting trade friction between the U.S. And Japan has raised fears among many of Asia exporting nations that the row could inflict far-reaching economic damage, businessmen and officials said. They told Reuter correspondents in Asian capitals a U.S. Move against Japan might boost protectionist sentiment in the U.S. And lead to curbs on American imports of their products. But some exporters said that while the conflict would hurt them in the long-run, in the short-term Tokyo loss might be their gain. The U.S. Has said it will impose 300 mln dlrs of tariffs on imports of Japanese electronics goods on April 17, in retaliation for Japan alleged failure to stick to a pact not to sell semiconductors on world markets at below cost. Unofficial Japanese estimates put the impact of the tariffs at 10 billion dlrs and spokesmen for major electronics firms said they would virtually halt exports of products hit by the new taxes. "We wouldn be able to do business," said a spokesman for leading Japa

spacy. load() is a convenience wrapper that reads the pipeline's config. cfg , uses the language and pipeline information to construct a Language object, loads in the model data and weights, and returns it.

In [14]:
def text_preprocessor(text):
    text = text.lower()
    
    return text
print(text_preprocessor("LOL"))


lol


In [15]:
def  accented_char_removal(text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text

normalized_text = accented_char_removal(normalized_text)

In [16]:
def expand_contractions(text, contraction_mapping=CONTRACTION_MAP):
    
    contractions_pattern = re.compile('({})'.format('|'.join(contraction_mapping.keys())), 
                                      flags=re.IGNORECASE|re.DOTALL)
    def expand_match(contraction):
        match = contraction.group(0)
        first_char = match[0]
        expanded_contraction = contraction_mapping.get(match)\
                                if contraction_mapping.get(match)\
                                else contraction_mapping.get(match.lower())                       
        expanded_contraction = first_char+expanded_contraction[1:]
        return expanded_contraction
        
    expanded_text = contractions_pattern.sub(expand_match, text)
    expanded_text = re.sub("'", "", expanded_text)
    return expanded_text

normalized_text = expand_contractions(normalized_text)
normalized_text


'Mounting trade friction between the U.S. And Japan has raised fears among many of Asia exporting nations that the row could inflict far-reaching economic damage, businessmen and officials said. They told Reuter correspondents in Asian capitals a U.S. Move against Japan might boost protectionist sentiment in the U.S. And lead to curbs on American imports of their products. But some exporters said that while the conflict would hurt them in the long-run, in the short-term Tokyo loss might be their gain. The U.S. Has said it will impose 300 mln dlrs of tariffs on imports of Japanese electronics goods on April 17, in retaliation for Japan alleged failure to stick to a pact not to sell semiconductors on world markets at below cost. Unofficial Japanese estimates put the impact of the tariffs at 10 billion dlrs and spokesmen for major electronics firms said they would virtually halt exports of products hit by the new taxes. "We wouldn be able to do business," said a spokesman for leading Japa

In [17]:
def lemmatize_text(text):
    doc = nlp(text)
    lemmatized_list = []
    for token in doc:
        lemmatized_list.append(token.lemma_)
    
    return lemmatized_list


len(lemmatize_text(normalized_text))



798

In [18]:
def tokenize_text(text):
    doc = nlp(text)
    tokenized_list = [token.text for token in doc]
    return tokenized_list
print(tokenize_text(u"Apples and oranges are similar. Boots and hippos aren't."))

['Apples', 'and', 'oranges', 'are', 'similar', '.', 'Boots', 'and', 'hippos', 'are', "n't", '.']


In [19]:
all_stopwords = nlp.Defaults.stop_words
"," in all_stopwords


False

In [20]:
def stopword_removal(text):
    all_stopwords = nlp.Defaults.stop_words
    
    #all_stopwords.remove('not')

    text_tokens = lemmatize_text(text)
    tokens_without_sw = [word for word in text_tokens if not word in all_stopwords]

    print(tokens_without_sw)

stopword_removal("The, and, if are stopwords, computer is not"  )

[',', ',', 'stopword', ',', 'computer']


In [23]:
def normalize_docs(documents):

    normalized_documents = []
    for doc in documents:
        doc = expand_contractions(doc)
        doc = ' '.join(re.sub(' +', ' ',doc.replace('\'s','').replace('\'t','').replace('&lt;','').replace(">","")).split('\n')[1:]).strip().replace('  ',' ')
        doc = expand_contractions(doc)
        normalized_documents.append(doc)
    return normalized_documents



    
list_normalized_text = normalize_docs(text_list)

df['text'] = list_normalized_text






In [27]:
df.iloc[0,2].split('  ')

['Mounting trade friction between the U.S. And Japan has raised fears among many of Asias exporting nations that the row could inflict far-reaching economic damage, businessmen and officials said. They told Reuter correspondents in Asian capitals a U.S. Move against Japan might boost protectionist sentiment in the U.S. And lead to curbs on American imports of their products. But some exporters said that while the conflict would hurt them in the long-run, in the short-term Tokyos loss might be their gain. The U.S. Has said it will impose 300 mln dlrs of tariffs on imports of Japanese electronics goods on April 17, in retaliation for Japans alleged failure to stick to a pact not to sell semiconductors on world markets at below cost. Unofficial Japanese estimates put the impact of the tariffs at 10 billion dlrs and spokesmen for major electronics firms said they would virtually halt exports of products hit by the new taxes. "We would not be able to do business," said a spokesman for leadi

In [29]:
df.to_excel('../Assets/preprocessed_text.xlsx',index=False)

In [5]:
pd.read_excel('../Assets/preprocessed_text.xlsx')

,ids,categories,text
0,test/14826,['trade'],Mounting trade friction between the U.S. And J...
1,test/14828,['grain'],A survey of 19 provinces and seven cities show...
2,test/14829,"['crude', 'nat-gas']",The Ministry of International Trade and Indust...
3,test/14832,"['corn', 'grain', 'rice', 'rubber', 'sugar', '...",Thailands trade deficit widened to 4.5 billion...
4,test/14833,"['palm-oil', 'veg-oil']",Indonesia expects crude palm oil (CPO) prices ...
...,...,...,...
10783,training/999,"['interest', 'money-fx']",The Bank of England said it had revised its fo...
10784,training/9992,['earn'],Qtly div 25 cts vs 25 cts prior Pay April 13 R...
10785,training/9993,['earn'],Qtly div 12 cts vs 12 cts prior Pay April 21 R...
10786,training/9994,['earn'],"Shr loss six cts vs loss 18 cts Net loss 89,47..."


In [ ]:

# def remove_special_characters(text, remove_digits=False):
#     pattern = r'[^a-zA-z0-9\s]' if not remove_digits else r'[^a-zA-z\s]'
#     text = re.sub(pattern, '', text)
#     return text

# remove_special_characters("Well this was fun! What do you think?", 
#                           remove_digits=True)